In [9]:

import numpy as np
import pandas as pd 

class Linear:
    def __init__(self,in_features,out_features):
        self.in_features = in_features
        self.out_features = out_features
        self.weights = np.random.randn(in_features, out_features) * 0.01
        self.biases = np.zeros((1, out_features))
        self.input = None
        self.grad_weights = None
        self.grad_biases = None
    def forward(self,x):
        self.input = x
        return np.dot(x, self.weights) + self.biases
    def backward(self,grad_output):
        self.grad_weights = np.dot(self.input.T, grad_output)
        self.grad_biases = np.sum(grad_output, axis=0, keepdims=True)
        return np.dot(grad_output, self.weights.T)        


class ReLU:
    def forward(self,x):
        self.input = x
        return np.maximum(0, x)
    def backward(self,grad_output):
        grad_input = grad_output.copy()
        grad_input[self.input <= 0] = 0
        return grad_input

class Sigmoid:
    def forward(self,x):
        self.input = x
        return 1 / (1 + np.exp(-x))
    def backward(self,grad_output):
        sigmoid = 1 / (1 + np.exp(-self.input))
        return grad_output * sigmoid * (1 - sigmoid)

class Tanh:
    def forward(self,x):
        self.input = x
        return np.tanh(x)
    def backward(self,grad_output):
        return grad_output * (1 - np.tanh(self.input) ** 2)

class Softmax:
    def forward(self,x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)
    def backward(self,grad_output):
        return grad_output

class CrossEntropyLoss:
    def forward(self,predictions,targets):
        m = predictions.shape[0]
        p = Softmax().forward(predictions)
        log_likelihood = -np.log(p[range(m), targets])
        loss = np.sum(log_likelihood) / m
        return loss
    def backward(self,predictions,targets):
        m = predictions.shape[0]
        grad = Softmax().forward(predictions)
        grad[range(m), targets] -= 1
        grad = grad / m
        return grad

class MSELoss:
    def forward(self,predictions,targets):
        return np.mean((predictions - targets) ** 2)
    def backward(self,predictions,targets):
        return 2 * (predictions - targets) / targets.size

class SGD:
    def __init__(self,parameters, learning_rate=0.01):
        self.parameters = parameters
        self.learning_rate = learning_rate
    def step(self):
        for param in self.parameters:
            param['weight'] -= self.learning_rate * param['grad_weight']
            param['bias'] -= self.learning_rate * param['grad_bias']

class Model:
    def __init__(self):
        self.layers = []
        self.loss_fn = None
        self.optimizer = None
    
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def compile(self, loss, optimizer):
        self.loss_fn = loss
        self.optimizer = optimizer

    def train(self, x_train, y_train, epochs=20):
        for epoch in range(epochs):
            # Forward pass
            predictions = x_train
            for layer in self.layers:
                predictions = layer.forward(predictions)

            # Compute loss
            loss = self.loss_fn.forward(predictions, y_train)
            print(f'Epoch {epoch + 1}, Loss: {loss}')

            # Backward pass
            self.loss_fn.backward(predictions, y_train)
            for layer in reversed(self.layers):
                layer.backward()

            # Update parameters
            self.optimizer.step()


    def predict(self,x):
        output = x
        for layer in self.layers:
            output = layer.forward(output)
        return output
    def evaluate(self, x_test, y_test):
        predictions = self.predict(x_test)
        loss = self.loss_fn.forward(predictions, y_test)
        accuracy = np.mean(np.argmax(predictions, axis=1) == y_test)
        print(f'Loss: {loss}, Accuracy: {accuracy}')
        return loss, accuracy
    def save(self,path):
        model_params = {
            "layers": self.layers,
            "optimizer": self.optimizer
        }
        np.save(path, model_params)
    def load(self,path):
        model_params = np.load(path, allow_pickle=True).item()
        self.layers = model_params['layers']
        self.optimizer = model_params['optimizer']        








